In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
# Read the csv file
df = pd.read_csv('data.csv', header=None)

# Remove empty rows
df = df.replace(np.nan,0)

# Convert the dataframe to a numpy array
data = df.to_numpy()

In [ ]:
data

In [ ]:
data.shape

In [ ]:
Mapping = {'R':1,'P':2,'S':3}

def map_values(x):
    return Mapping.get(x, x)

In [ ]:
vectorized_map_values = np.vectorize(map_values)
converted_data = vectorized_map_values(data)

In [ ]:
converted_data.dtype

In [ ]:
segments = []

for i in range(0, converted_data.shape[1] - 20):
    segment = converted_data[0][i:i+20]
    segments.append(segment)


In [ ]:
segments

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(converted_data)

In [ ]:
BATCH_SIZE = 20
batched_data= dataset.batch(BATCH_SIZE+1, drop_remainder=True)

In [ ]:
def split_input_target(chunk):  
    input_text = chunk[:-1]  
    target_text = chunk[1:]  
    return input_text, target_text

dataset = batched_data.map(split_input_target)

In [ ]:
dataset

In [ ]:
batched_data


In [ ]:
data = dataset.shuffle(1000).batch(20,drop_remainder = True)

In [ ]:
data

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

VOCAB_SIZE = 4
EMBEDDING_DIM = 32
RNN_UNITS = 1024

In [ ]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(data,epochs = 10) 
#next attempt check what datasets can go into this function, make sure I am puttin the right stuff

In [ ]:
model.input_shape